## 基于GLM的文本摘要

### Step1 导入相关包

In [2]:
import os

# 设置可见的 GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

from datasets import  Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

### Step2 加载数据集

In [3]:
ds = Dataset.load_from_disk("nlpcc_2017")
ds

Dataset({
    features: ['title', 'content'],
    num_rows: 5000
})

In [4]:
ds = ds.train_test_split(200, seed=42)
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 4800
    })
    test: Dataset({
        features: ['title', 'content'],
        num_rows: 200
    })
})

In [5]:
ds["train"][0]

{'title': '郴州市发布雷电橙色预警:过去2小时北湖区、苏仙区、郴州市区、桂阳县、宜章县、嘉禾县、资兴市、桂东县、汝城县已经受...',
 'content': '发布日期:2015-03-3007:55:33郴州市气象台3月30日7时52分发布雷电橙色预警信号:过去2小时北湖区、苏仙区、郴州市区、桂阳县、宜章县、嘉禾县、资兴市、桂东县、汝城县已经受雷电活动影响,并将持续,出现雷电灾害事故的可能性比较大,请注意防范。图例标准防御指南2小时内发生雷电活动的可能性很大,或者已经受雷电活动影响,且可能持续,出现雷电灾害事故的可能性比较大。1、政府及相关部门按照职责落实防雷应急措施;2、人员应当留在室内,并关好门窗;3、户外人员应当躲入有防雷设施的建筑物或者汽车内;4、切断危险电源,不要在树下、电杆下、塔吊下避雨;5、在空旷场地不要打伞,不要把农具、羽毛球拍、高尔夫球杆等扛在肩上。'}

### Step3 数据集处理

In [6]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/glm-large-chinese", trust_remote_code=True)
tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GLMChineseTokenizer(name_or_path='THUDM/glm-large-chinese', vocab_size=50000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='left', special_tokens={'eos_token': '<|endoftext|>', 'unk_token': '[UNK]', 'pad_token': '<|endoftext|>', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['<|startofpiece|>', '<|endofpiece|>', '[gMASK]', '[sMASK]']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50000: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50001: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50002: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50003: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50004: AddedToken("[UNUSED1]", rstrip=False, lstrip=False, single_word=Fa

In [7]:
def process_function(examples):
    contents = ["摘要生成：\n" + e + tokenizer.mask_token for e in examples["content"]]  # 加个[MASK]告诉他从这开始生成
    inputs = tokenizer(contents, max_length=384, truncation=True, padding="max_length", return_tensors="pt")
    inputs = tokenizer.build_inputs_for_generation(inputs, targets=examples["title"], max_gen_length=64, padding=True)
    return inputs

In [8]:
tokenized_ds = ds.map(process_function, batched=True, remove_columns=ds["train"].column_names)
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'position_ids', 'attention_mask', 'labels'],
        num_rows: 4800
    })
    test: Dataset({
        features: ['input_ids', 'position_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})

In [9]:
tokenizer.decode(tokenized_ds["train"][0]["input_ids"])

'[CLS] 摘要生成: 发布日期:2015-03-3007:55:33郴州市气象台3月30日7时52分发布雷电橙色预警信号:过去2小时北湖区、苏仙区、郴州市区、桂阳县、宜章县、嘉禾县、资兴市、桂东县、汝城县已经受雷电活动影响,并将持续,出现雷电灾害事故的可能性比较大,请注意防范。图例标准防御指南2小时内发生雷电活动的可能性很大,或者已经受雷电活动影响,且可能持续,出现雷电灾害事故的可能性比较大。1、政府及相关部门按照职责落实防雷应急措施;2、人员应当留在室内,并关好门窗;3、户外人员应当躲入有防雷设施的建筑物或者汽车内;4、切断危险电源,不要在树下、电杆下、塔吊下避雨;5、在空旷场地不要打伞,不要把农具、羽毛球拍、高尔夫球杆等扛在肩上。[MASK]<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoft

In [10]:
# tokenizer.decode(tokenized_ds["train"][0]["labels"]) # 有-100，会报错
print(tokenized_ds["train"][0]["labels"])

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

In [11]:
print(tokenized_ds["train"][0]["position_ids"])

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221

### Step4 创建模型

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained("THUDM/glm-large-chinese", trust_remote_code=True)

### Step5 配置训练参数

In [13]:
import logging

logging.basicConfig(level=logging.INFO)

In [17]:
args = Seq2SeqTrainingArguments(
    output_dir="./summary-glm",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1
)

### Step6 创建训练器

In [18]:
trainer = Seq2SeqTrainer(
    args=args,
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds["train"],
)

### Step7 模型训练

In [19]:
trainer.train()

Step,Training Loss
10,1.027600
20,1.725800
30,1.796300
40,1.697400
50,1.681600
60,1.634900
70,1.675200
80,1.658400
90,1.721600
100,1.564900


TrainOutput(global_step=150, training_loss=1.602557455698649, metrics={'train_runtime': 1049.3961, 'train_samples_per_second': 4.574, 'train_steps_per_second': 0.143, 'total_flos': 4561779975782400.0, 'train_loss': 1.602557455698649, 'epoch': 1.0})

### Step8 模型推理

In [28]:
input_text = ds["train"][-1]["content"]
inputs = tokenizer("摘要生成：\n" + input_text + tokenizer.mask_token, return_tensors="pt")
inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=64)
inputs = inputs.to("cuda")
output = model.generate(**inputs, max_new_tokens=64, eos_token_id=tokenizer.eop_token_id)
tokenizer.decode(output[0].tolist())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


'[CLS] 摘要生成: 网传微博截图人民网北京7月28日电(田雪)今日,网上流传多张图片称“醉酒女子被流浪汉强奸后自杀”引起网民热议。经公安机关查证,此事系7月27日凌晨发生在广西崇左市,该女子精神失常,犯罪嫌疑人钟某对强奸犯罪事实供认不讳。微博上有网民发图文求证“福州一农业银行内,醉酒女子被流浪汉强奸后自杀”一事,导致大量网民围观。记者查证发现,该图文系昨日发生在广西崇左市的一起案件。崇左市公安局江州分局官方微博通报称:7月27日凌晨3时许,一男子与一女子在崇左市江州区新民路红绿灯附近农业银行自动取款机大厅内发生性关系。经调查了解,该女子精神失常,姓名、地址不详。犯罪嫌疑人钟某对强奸犯罪事实供认不讳。目前,受害人已得到救助,案件在进一步调查当中。而记者搜索发现,同样的图文在多地流传,博人眼球,嘉兴、合肥、温州、宁波等地纷纷“躺枪”。“江州”公安微博截图[MASK]<|endoftext|> <|startofpiece|> 网传“崇左一女子被流浪汉强奸后自杀”引发热议,经查,该女子精神失常,犯罪嫌疑人供认不讳。 <|endofpiece|>'

In [27]:
input_text = ds["test"][-1]["content"]
inputs = tokenizer("摘要生成：\n" + input_text + tokenizer.mask_token, return_tensors="pt")
inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=64)
inputs = inputs.to("cuda")
output = model.generate(**inputs, max_new_tokens=64, eos_token_id=tokenizer.eop_token_id, do_sample=True)
tokenizer.decode(output[0].tolist())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


'[CLS] 摘要生成: 据英国《镜报》8月3日报道,美国弗吉尼亚州的男子杰森·斯坦福(JasonStandford)同意妻子前往火星的单程旅程,旅程将于2026年正式开始。这次旅程的目的是为了在火星上开拓人类的居住地,为人类赢得更多的生存空间。据悉,“火星一号”计划由荷兰企业家巴斯?兰多普(BasLansdorp)于2012年5月宣布启动。杰森的妻子索菲亚·范·米特(SoniaVanMeter)志愿成为“火星一号”计划的一员,将于2026年前往火星,并不再返航。杰森和他的两个儿子将在11年后与索菲亚说永远的再见。杰森说:“这项任务的意义远大于我对索菲亚的思念,这次世界上最大的冒险也许会改变我的生活,但更多的是改变所有人的生活。如果我们成功地在火星上建立起自己的居住地,这将是划时代的里程碑。”杰森和他的两个儿子将在11年后与索菲亚说永远的再见。[MASK]<|endoftext|> <|startofpiece|> 美国男子同意前往火星,往返时间定在2026年,他的妻子为“火星一号”计划的一员,将11年后与索菲亚说永远的再见。 <|endofpiece|>'

In [34]:
import torch

model.eval()

def predict_test():
    predict = []
    with torch.inference_mode():
        for d in ds["test"]:
            inputs = tokenizer("摘要生成：\n" + d["content"] + tokenizer.mask_token, return_tensors="pt")
            inputs = tokenizer.build_inputs_for_generation(inputs, max_gen_length=64)
            inputs = inputs.to("cuda")
            output = model.generate(**inputs, max_new_tokens=64, eos_token_id=tokenizer.eop_token_id)
            predict.append(tokenizer.decode(output[0].tolist()).split("<|startofpiece|>")[1].replace("<|endofpiece|>", "").strip())
            print("curID:", len(predict))
    return predict

In [35]:
result = predict_test()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


curID: 1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


curID: 2


...

curID: 197


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


curID: 198


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50007 for open-end generation.


curID: 199
curID: 200


In [36]:
from rouge_chinese import Rouge

rouge = Rouge()

decode_predicts = [" ".join(p) for p in result]
decode_labels = [" ".join(l) for l in ds["test"]["title"]]
scores = rouge.get_scores(decode_predicts, decode_labels, avg=True)

{
    "rouge-1": scores["rouge-1"]["f"],
    "rouge-2": scores["rouge-2"]["f"],
    "rouge-l": scores["rouge-l"]["f"]
}

{'rouge-1': 0.570047509741109,
 'rouge-2': 0.3860718940946789,
 'rouge-l': 0.47900242596542886}